# Model Evaluation

In [1]:
import polars as pl

actual_events = pl.read_parquet('../input/comp9417-preprocessing-and-test-data/actual_events.parquet')
test_df = pl.read_parquet('../input/comp9417-preprocessing-and-test-data/test_df.parquet')

In [2]:
id2type = {0: 'clicks', 1: 'carts', 2: 'orders'}
type2id = {'clicks': 0, 'carts': 1, 'orders': 2}

In [3]:
import pickle

def load_model(name):
    with open(name, 'rb') as file:  
        model = pickle.load(file)
        
    return model

In [4]:
import pandas as pd

def kaggle_evaluate(name, pred_df):
#     print(pred_df, actual_events)
    click_preds = pred_df.groupby('session').agg(pl.col('aid').sort_by('click_scores', descending=True).slice(0,20))
    cart_preds = pred_df.groupby('session').agg(pl.col('aid').sort_by('cart_scores', descending=True).slice(0,20))
    order_preds = pred_df.groupby('session').agg(pl.col('aid').sort_by('order_scores', descending=True).slice(0,20))

    click_new = click_preds.with_columns(
        pl.lit('clicks').alias('type')
    ).rename({'aid':'pred_labels'})
    cart_new = cart_preds.with_columns(
        pl.lit('carts').alias('type')
    ).rename({'aid':'pred_labels'})
    order_new = order_preds.with_columns(
        pl.lit('orders').alias('type')
    ).rename({'aid':'pred_labels'})

    preds = pl.concat([click_new, cart_new, order_new])
    print(preds)
    print(actual_events)
    
    gt = actual_events.groupby(['session', 'type']).agg(pl.col('aid')).rename({'aid': 'gt_labels'}).sort(by='session').with_columns(pl.col('type').map_dict(id2type))
    gt = gt.to_pandas()
    gt.loc[gt.type == 'clicks', 'gt_labels'] = gt.loc[gt.type == 'clicks', 'gt_labels'].str[:1]
    gt = pl.from_pandas(gt)
    
    preds_and_gt = gt.join(preds, how='left', on=['session', 'type']).with_columns(
        hits = pl.col('pred_labels').list.intersection('gt_labels').list.lengths(),
        gt_count = pl.col('gt_labels').list.lengths()
    )
    
    preds_and_gt = preds_and_gt.to_pandas()
    preds_and_gt.loc[preds_and_gt.type == 'carts', 'gt_labels'] = preds_and_gt.loc[preds_and_gt.type == 'carts', 'gt_labels'].str[:20]
    preds_and_gt.loc[preds_and_gt.type == 'orders', 'gt_labels'] = preds_and_gt.loc[preds_and_gt.type == 'orders', 'gt_labels'].str[:20]
    preds_and_gt = pl.from_pandas(preds_and_gt)
    preds_and_gt = preds_and_gt.with_columns(
        gt_count = pl.col('gt_labels').list.lengths()
    )
    
    recall_per_type = preds_and_gt.groupby('type').agg(recall = pl.col('hits').sum() / pl.col('gt_count').sum())
    local_validation_score = 0
    weights = {'clicks': 0.1, 'carts': 0.3, 'orders': 0.6}
    for row in recall_per_type.rows(named=True):
        local_validation_score += row['recall'] * weights[row['type']]
    
    print(f'{name}: {local_validation_score}')

In [5]:
def evaluate_model(name, models):
    model_click = models[0]
    model_cart = models[1]
    model_order = models[2]
    
    click_scores = model_click.predict(test_df.to_pandas())
    cart_scores = model_cart.predict(test_df.to_pandas())
    order_scores = model_order.predict(test_df.to_pandas())

    pred_df = test_df.with_columns(click_scores = pl.lit(click_scores), cart_scores = pl.lit(cart_scores), order_scores = pl.lit(order_scores))
    
    kaggle_evaluate(name, pred_df)

In [6]:
def kaggle_evaluate2(name, preds):
    gt = actual_events.groupby(['session', 'type']).agg(pl.col('aid')).rename({'aid': 'gt_labels'}).sort(by='session').with_columns(pl.col('type').map_dict(id2type))
    gt = gt.to_pandas()
    gt.loc[gt.type == 'clicks', 'gt_labels'] = gt.loc[gt.type == 'clicks', 'gt_labels'].str[:1]
    gt = pl.from_pandas(gt)
    
    preds_and_gt = gt.join(preds, how='left', on=['session', 'type']).with_columns(
        hits = pl.col('pred_labels').list.intersection('gt_labels').list.lengths(),
        gt_count = pl.col('gt_labels').list.lengths()
    )
    
    preds_and_gt = preds_and_gt.to_pandas()
    preds_and_gt.loc[preds_and_gt.type == 'carts', 'gt_labels'] = preds_and_gt.loc[preds_and_gt.type == 'carts', 'gt_labels'].str[:20]
    preds_and_gt.loc[preds_and_gt.type == 'orders', 'gt_labels'] = preds_and_gt.loc[preds_and_gt.type == 'orders', 'gt_labels'].str[:20]
    preds_and_gt = pl.from_pandas(preds_and_gt)
    preds_and_gt = preds_and_gt.with_columns(
        gt_count = pl.col('gt_labels').list.lengths()
    )
    
    recall_per_type = preds_and_gt.groupby('type').agg(recall = pl.col('hits').sum() / pl.col('gt_count').sum())
    local_validation_score = 0
    weights = {'clicks': 0.1, 'carts': 0.3, 'orders': 0.6}
    for row in recall_per_type.rows(named=True):
        local_validation_score += row['recall'] * weights[row['type']]
    
    print(f'{name}: {local_validation_score}')

In [7]:
# Simple candidate generator 
simple_preds = test_df.select(['session', 'aid']).groupby('session').agg(pred_labels=pl.col('aid'))
simple_preds = pl.concat([simple_preds.with_columns(pl.lit('click').alias('type')), simple_preds.with_columns(pl.lit('carts').alias('type')), simple_preds.with_columns(pl.lit('orders').alias('type'))])
print(simple_preds)
kaggle_evaluate2('simple_candidate_gen', simple_preds)

shape: (600_000, 3)
┌──────────┬──────────────────────────┬────────┐
│ session  ┆ pred_labels              ┆ type   │
│ ---      ┆ ---                      ┆ ---    │
│ i32      ┆ list[i32]                ┆ str    │
╞══════════╪══════════════════════════╪════════╡
│ 12978436 ┆ [863998]                 ┆ click  │
│ 13004644 ┆ [1654098]                ┆ click  │
│ 12930160 ┆ [1733797]                ┆ click  │
│ 12970716 ┆ [1274934]                ┆ click  │
│ …        ┆ …                        ┆ …      │
│ 12973983 ┆ [95232, 554660, … 78453] ┆ orders │
│ 12917691 ┆ [1056655]                ┆ orders │
│ 12937159 ┆ [1350261]                ┆ orders │
│ 13015075 ┆ [554660]                 ┆ orders │
└──────────┴──────────────────────────┴────────┘
simple_candidate_gen: 0.415829379622464


In [8]:
event_types = ['click', 'cart', 'order']
models = []
for event in event_types:
    models.append(load_model(f'../input/comp9417-training-xgboost/xgb_{event}.pkl'))

evaluate_model('XGB', models)    

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


shape: (600_000, 3)
┌──────────┬─────────────────────────────┬────────┐
│ session  ┆ pred_labels                 ┆ type   │
│ ---      ┆ ---                         ┆ ---    │
│ i32      ┆ list[i32]                   ┆ str    │
╞══════════╪═════════════════════════════╪════════╡
│ 12982584 ┆ [1324525]                   ┆ clicks │
│ 13046604 ┆ [1167765, 554660]           ┆ clicks │
│ 12984900 ┆ [1132033, 572241]           ┆ clicks │
│ 13027040 ┆ [335398, 341862, … 1481038] ┆ clicks │
│ …        ┆ …                           ┆ …      │
│ 12951275 ┆ [1577268]                   ┆ orders │
│ 12952147 ┆ [133627, 1232342]           ┆ orders │
│ 12982631 ┆ [1332057]                   ┆ orders │
│ 12913511 ┆ [380509, 69440, … 1569250]  ┆ orders │
└──────────┴─────────────────────────────┴────────┘
shape: (924_654, 4)
┌──────────┬─────────┬────────────┬──────┐
│ session  ┆ aid     ┆ ts         ┆ type │
│ ---      ┆ ---     ┆ ---        ┆ ---  │
│ i32      ┆ i32     ┆ i32        ┆ i8   │
╞═══════

In [9]:
event_types = ['click', 'cart', 'order']
models = []
for event in event_types:
    models.append(load_model(f'../input/comp9417-training-lightgbm/lgbm_{event}.pkl'))

evaluate_model('LightGBM', models)    

shape: (600_000, 3)
┌──────────┬─────────────────────────────┬────────┐
│ session  ┆ pred_labels                 ┆ type   │
│ ---      ┆ ---                         ┆ ---    │
│ i32      ┆ list[i32]                   ┆ str    │
╞══════════╪═════════════════════════════╪════════╡
│ 12901692 ┆ [325854, 96165, … 1133328]  ┆ clicks │
│ 12973128 ┆ [1119434, 1744977, 1384241] ┆ clicks │
│ 13011380 ┆ [1501145]                   ┆ clicks │
│ 12911348 ┆ [1451391, 1110278]          ┆ clicks │
│ …        ┆ …                           ┆ …      │
│ 12918963 ┆ [420680, 216633, … 1098423] ┆ orders │
│ 12914203 ┆ [738508]                    ┆ orders │
│ 12965095 ┆ [1726560]                   ┆ orders │
│ 13050023 ┆ [1299205]                   ┆ orders │
└──────────┴─────────────────────────────┴────────┘
shape: (924_654, 4)
┌──────────┬─────────┬────────────┬──────┐
│ session  ┆ aid     ┆ ts         ┆ type │
│ ---      ┆ ---     ┆ ---        ┆ ---  │
│ i32      ┆ i32     ┆ i32        ┆ i8   │
╞═══════